In [739]:
import pandas as pd
import numpy as np
import math

In [23]:
alunosUFCG = pd.read_csv('../alunosUFCGAnon.csv', thousands = ',')

alunosUFCG.head()

,Matricula,Cod_Curso,Cod_Disciplina,Cod_Evasao,Nome_Curso,Periodo,Nome_Disciplina,Creditos,Departamento,Media_Disciplina,Situacao,Centro,Campus,Creditos_Curso,Tipo,Periodo_Ingresso,Periodo_Relativo
0,B7350,14102100,1411191,0,CIÊNCIA DA COMPUTAÇÃO - D,2004.2,INTERCONEXÃO DE REDES DE COMPUTADORES,2,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,8.5,Aprovado,14,1,208.0,Obrigatória,2002.1,6
1,B7350,14102100,1411196,0,CIÊNCIA DA COMPUTAÇÃO - D,2005.1,INTELIGENCIA ARTIFICIAL I,4,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,9.3,Aprovado,14,1,208.0,Obrigatória,2002.1,7
2,B7350,14102100,1411255,0,CIÊNCIA DA COMPUTAÇÃO - D,2005.1,TECC(LAB DE SISTEMAS OPERACIONAIS),2,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,10.0,Aprovado,14,1,208.0,Optativa,2002.1,7
3,B7350,14102100,1411173,0,CIÊNCIA DA COMPUTAÇÃO - D,2003.2,LÓGICA MATEMÁTICA,4,UNID. ACAD. DE SISTEMAS E COMPUTAÇÃO,8.7,Aprovado,14,1,208.0,Obrigatória,2002.1,4
4,B7350,14102100,1109035,0,CIÊNCIA DA COMPUTAÇÃO - D,2002.1,ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA,4,UNID. ACAD. DE MATEMÁTICA,9.3,Aprovado,14,1,208.0,Obrigatória,2002.1,1


In [24]:
cadeiras_obrigatorias_df = alunosUFCG[alunosUFCG.Tipo == "Obrigatória"]
cadeiras_obrigatorias_filtradas = cadeiras_obrigatorias_df[['Matricula', 'Cod_Disciplina', 'Nome_Disciplina', 'Periodo', 'Creditos', 'Media_Disciplina', 'Situacao', 'Periodo_Ingresso', 'Periodo_Relativo', 'Nome_Curso']]

In [357]:
# selecionando os cursos


COMPUTACAOdf = cadeiras_obrigatorias_filtradas[cadeiras_obrigatorias_df.Nome_Curso =="CIÊNCIA DA COMPUTAÇÃO - D"]
COMPUTACAOtdf = pd.pivot_table(COMPUTACAOdf, values = 'Media_Disciplina', index = ['Nome_Disciplina'], columns = 'Matricula')
# COMPUTACAOdf = COMPUTACAOdf.fillna(-1)
# del COMPUTACAOdf['Nome_Curso']
print(COMPUTACAOdf)
print(">--------------------------------<")

# FISICA_Bdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("FÍSICA \(BAC\)")]
# FISICA_Bdf = pandas.pivot_table(FISICA_Bdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# # del FISICA_Bdf['Nome_Curso']
# print(FISICA_Bdf.head())
# print(">--------------------------------<")

# MATEMATICA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("MATEMÁTICA \(LIC\)")]
# MATEMATICA_Ldf = pandas.pivot_table(MATEMATICA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# # del MATEMATICA_Ldf['Nome_Curso']
# print(MATEMATICA_Ldf.head())
# print(">--------------------------------<")

# MATEMATICA_Bdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("MATEMÁTICA \(BAC\)")]
# MATEMATICA_Bdf = pandas.pivot_table(MATEMATICA_Bdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MATEMATICA_Ldf.head())
# print(">--------------------------------<")

# METEOROLOGIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("METEOROLOGIA")]
# METEOROLOGIAdf = pandas.pivot_table(METEOROLOGIAdf, values = 'Media_Disciplina', index = 'Matricula', columns = 'Nome_Disciplina')
# print(METEOROLOGIAdf.head())
# print(">--------------------------------<")

# AGRICOLAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA AGRÍCOLA")]
# AGRICOLAdf = pandas.pivot_table(AGRICOLAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(AGRICOLAdf.head())
# print(">--------------------------------<")

# CIVILdf =  notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA CIVIL")]
# CIVILdf = pandas.pivot_table(CIVILdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(CIVILdf.head())
# print(">--------------------------------<")

# ELETdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA ELÉTRICA")]
# ELETdf = pandas.pivot_table(ELETdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ELETdf.head())
# print(">--------------------------------<")

# MECANICAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA MECÂNICA")]
# MECANICAdf = pandas.pivot_table(MECANICAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MECANICAdf.head())
# print(">--------------------------------<")

# MATERIAISdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA MATERIAIS")]
# MATERIAISdf = pandas.pivot_table(MATERIAISdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MATERIAISdf.head())
# print(">--------------------------------<")

# MINASdf =  notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA DE MINAS")]
# MINASdf = pandas.pivot_table(MINASdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MINASdf.head())
# print(">--------------------------------<")

# ENG_QUIMICAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA QUÍMICA")]
# ENG_QUIMICAdf = pandas.pivot_table(ENG_QUIMICAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ENG_QUIMICAdf.head())
# print(">--------------------------------<")

# MEDICINAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("MEDICINA")]
# MEDICINAdf = pandas.pivot_table(MEDICINAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MEDICINAdf.head())
# print(">--------------------------------<")

# ADMINISTRACAOdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ADMINISTRAÇÃO")]
# ADMINISTRACAOdf = pandas.pivot_table(ADMINISTRACAOdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ADMINISTRACAOdf.head())
# print(">--------------------------------<")

# ECONOMICASdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("CIÊNCIAS ECONÔMICAS")]
# ECONOMICASdf = pandas.pivot_table(ECONOMICASdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ECONOMICASdf.head())
# print(">--------------------------------<")

# SOCIAIS_Bdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("CIÊNCIAS SOCIAIS \(BAC\)")]
# SOCIAIS_Bdf = pandas.pivot_table(SOCIAIS_Bdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(SOCIAIS_Bdf.head())
# print(">--------------------------------<")

# HISTORIA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("HISTÓRIA \(LIC\)")]
# HISTORIA_Ldf = pandas.pivot_table(HISTORIA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(HISTORIA_Ldf.head())
# print(">--------------------------------<")

# PEDAGOGIA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("PEDAGOGIA \(LIC\)")]
# PEDAGOGIA_Ldf = pandas.pivot_table(PEDAGOGIA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(PEDAGOGIA_Ldf.head())
# print(">--------------------------------<")

# DESIGNdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("DESIGN")]
# DESIGNdf = pandas.pivot_table(DESIGNdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(DESIGNdf.head())
# print(">--------------------------------<")

# SOCIAIS_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("CIÊNCIAS SOCIAIS \(LIC\)")]
# SOCIAIS_Ldf = pandas.pivot_table(SOCIAIS_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(SOCIAIS_Ldf.head())
# print(">--------------------------------<")

# ARTE_MIDIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ARTE E MÍDIA")]
# ARTE_MIDIAdf = pandas.pivot_table(ARTE_MIDIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ARTE_MIDIAdf.head())
# print(">--------------------------------<")

# PRODUÇÃOdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA DE PRODUÇÃO")]
# PRODUÇÃOdf =pandas.pivot_table(PRODUÇÃOdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(PRODUÇÃOdf.head())
# print(">--------------------------------<")

# ALIMENTOSdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA DE ALIMENTOS")]
# ALIMENTOSdf = pandas.pivot_table(ALIMENTOSdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ALIMENTOSdf.head())
# print(">--------------------------------<")

# PETROLEOdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA DE PETRÓLEO")]
# PETROLEOdf = pandas.pivot_table(PETROLEOdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(PETROLEOdf.head())
# print(">--------------------------------<")

# ENFERMAGEMdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENFERMAGEM")]
# ENFERMAGEMdf = pandas.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ENFERMAGEMdf.head())
# print(">--------------------------------<")

# FILOSOFIA_Bdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("FILOSOFIA \(BAC\)")]
# FILOSOFIA_Bdf = pandas.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(FILOSOFIA_Bdf.head())
# print(">--------------------------------<")

# GEOGRAFIA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("GEOGRAFIA \(LIC\)")]
# GEOGRAFIA_Ldf = pandas.pivot_table(GEOGRAFIA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(GEOGRAFIA_Ldf.head())
# print(">--------------------------------<")

# MUSICA_Bdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("MÚSICA \(BAC\)")]
# MUSICA_Bdf = pandas.pivot_table(MUSICA_Bdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MUSICA_Bdf.head())
# print(">--------------------------------<")

# MUSICA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("MÚSICA \(LIC\)")]
# MUSICA_Ldf = pandas.pivot_table(MUSICA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(MUSICA_Ldf.head())
# print(">--------------------------------<")

# INDIGENAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("INDÍGENA")]
# INDIGENAdf = pandas.pivot_table(INDIGENAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(INDIGENAdf.head())
# print(">--------------------------------<")

# ESTATISTICAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ESTATÍSTICA")]
# ESTATISTICAdf = pandas.pivot_table(ESTATISTICAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ESTATISTICAdf.head())
# print(">--------------------------------<")

# PSICOLOGIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("PSICOLOGIA")]
# PSICOLOGIAdf = pandas.pivot_table(PSICOLOGIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(PSICOLOGIAdf.head())
# print(">--------------------------------<")

# FILOSOFIA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("FILOSOFIA")]
# FILOSOFIA_Ldf = pandas.pivot_table(FILOSOFIA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(FILOSOFIA_Ldf.head())
# print(">--------------------------------<")

# ARQUITETURAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ARQUITETURA")]
# ARQUITETURAdf = pandas.pivot_table(ARQUITETURAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ARQUITETURAdf.head())
# print(">--------------------------------<")

# COMUNICACAOdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("COMUNICAÇÃO")]
# COMUNICACAOdf = pandas.pivot_table(COMUNICACAOdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(COMUNICACAOdf.head())
# print(">--------------------------------<")

# FISICA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("FÍSICA \(LIC\)")]
# FISICA_Ldf = pandas.pivot_table(FISICA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(FISICA_Ldf.head())
# print(">--------------------------------<")

# PORTUGUES_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("PORTUGUES \(LIC\)")]
# PORTUGUES_Ldf = pandas.pivot_table(PORTUGUES_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(PORTUGUES_Ldf.head())
# print(">--------------------------------<")

# PORT_FRANC_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("FRANC")]
# PORT_FRANC_Ldf = pandas.pivot_table(PORT_FRANC_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(PORT_FRANC_Ldf.head())
# print(">--------------------------------<")

# INGLES_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("INGLESA \(LIC\)")]
# INGLES_Ldf = pandas.pivot_table(INGLES_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(INGLES_Ldf.head())
# print(">--------------------------------<")

# ESPANHOL_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ESPANHOL \(LIC\)")]
# ESPANHOL_Ldf = pandas.pivot_table(ESPANHOL_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ESPANHOL_Ldf.head())
# print(">--------------------------------<")

# QUIMICA_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("QUÍMICA \(LIC\)")]
# QUIMICA_Ldf = pandas.pivot_table(QUIMICA_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(QUIMICA_Ldf.head())
# print(">--------------------------------<")

# BIOLOGICAS_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("CIÊNCIAS BIOLÓGICAS \(LIC\)")]
# BIOLOGICAS_Ldf = pandas.pivot_table(BIOLOGICAS_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(BIOLOGICAS_Ldf.head())
# print(">--------------------------------<")

# DIREITOdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("DIREITO")]
# DIREITOdf = pandas.pivot_table(DIREITOdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(DIREITOdf.head())
# print(">--------------------------------<")

# CONTABEISdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("CIÊNCIAS CONTÁBEIS")]
# CONTABEISdf = pandas.pivot_table(CONTABEISdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(CONTABEISdf.head())
# print(">--------------------------------<")

# SERVICO_SOCIALdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("SERVIÇO SOCIAL")]
# SERVICO_SOCIALdf = pandas.pivot_table(SERVICO_SOCIALdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(SERVICO_SOCIALdf.head())
# print(">--------------------------------<")

# ENG_FLORESTALdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA FLORESTAL")]
# ENG_FLORESTALdf = pandas.pivot_table(ENG_FLORESTALdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ENG_FLORESTALdf.head())
# print(">--------------------------------<")

# VETERINARIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("MEDICINA VETERINÁRIA")]
# VETERINARIAdf = pandas.pivot_table(VETERINARIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(VETERINARIAdf.head())
# print(">--------------------------------<")

# ODONTOLOGIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ODONTOLOGIA")]
# ODONTOLOGIAdf = pandas.pivot_table(ODONTOLOGIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ODONTOLOGIAdf.head())
# print(">--------------------------------<")

# FARMACIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("FÁRMACIA")]
# FARMACIAdf = pandas.pivot_table(FARMACIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(FARMACIAdf.head())
# print(">--------------------------------<")

# NUTRICAOdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("NUTRIÇÃO")]
# NUTRICAOdf = pandas.pivot_table(NUTRICAOdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(NUTRICAOdf.head())
# print(">--------------------------------<")

# ENG_BIOSSISTEMASdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA DE BIOSSISTEMAS")]
# ENG_BIOSSISTEMASdf = pandas.pivot_table(ENG_BIOSSISTEMASdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ENG_BIOSSISTEMASdf.head())
# print(">--------------------------------<")

# ENG_BIOTECNOLOGIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA DE BIOTECNOLOGIA E BIOPROCESSOS")]
# ENG_BIOTECNOLOGIAdf = pandas.pivot_table(ENG_BIOTECNOLOGIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(ENG_BIOTECNOLOGIAdf.head())
# print(">--------------------------------<")

# EDUCAÇÃO_CAMPO_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("EDUCAÇÃO DO CAMPO")]                                                                       
# EDUCAÇÃO_CAMPO_Ldf = pandas.pivot_table(EDUCAÇÃO_CAMPO_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(EDUCAÇÃO_CAMPO_Ldf.head())
# print(">--------------------------------<")

# EDUCAÇÃO_CAMPO_Ldf = notas_alunos[notas_alunos.Nome_Curso.str.contains("GESTÃO PÚBLICA")]
# EDUCAÇÃO_CAMPO_Ldf = pandas.pivot_table(EDUCAÇÃO_CAMPO_Ldf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(EDUCAÇÃO_CAMPO_Ldf.head())
# print(">--------------------------------<")

# TECN_AGROECOLOGIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("AGROTECNOLOGIA")]
# TECN_AGROECOLOGIAdf = pandas.pivot_table(TECN_AGROECOLOGIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(TECN_AGROECOLOGIAdf.head())
# print(">--------------------------------<")

# AMBIENTALdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("ENGENHARIA AMBIENTAL")]
# AMBIENTALdf = pandas.pivot_table(AMBIENTALdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(AMBIENTALdf.head())
# print(">--------------------------------<")

# AGRONOMIAdf = notas_alunos[notas_alunos.Nome_Curso.str.contains("AGRONOMIA")]
# AGRONOMIAdf = pandas.pivot_table(AGRONOMIAdf, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')
# print(AGRONOMIAdf.head())
# print(">--------------------------------<")


# pandas.pivot_table(notas_alunos, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')


       Matricula  Cod_Disciplina                           Nome_Disciplina  \
0          B7350         1411191     INTERCONEXÃO DE REDES DE COMPUTADORES   
1          B7350         1411196                 INTELIGENCIA ARTIFICIAL I   
3          B7350         1411173                         LÓGICA MATEMÁTICA   
4          B7350         1109035    ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA   
5          B7350         1411190                     REDES DE COMPUTADORES   
7          B7350         1305220                   INFORMÁTICA E SOCIEDADE   
8          B7350         1411169        PARADIGMAS DE LING. DE PROGRAMAÇÃO   
9          B7350         1411192                     SISTEMAS OPERACIONAIS   
11         B7350         1411181             LABORATÓRIO DE PROGRAMAÇÃO II   
13         B7350         1109103          CALCULO DIFERENCIAL E INTEGRAL I   
14         B7350         1411184  LAB.DE INTERCON.DE REDES DE COMPUTADORES   
15         B7350         1411171                      TEORIA DA 

In [27]:
print(COMPUTACAOdf.columns.values.tolist())

meu_historico = {'ALGEBRA LINEAR I':7.5, 'CALCULO DIFERENCIAL E INTEGRAL I':8.5, 'CALCULO DIFERENCIAL E INTEGRAL II':8.5, 'ESTRUTURA DE DADOS E ALGORITMOS':8.5, 'FUNDAMENTOS DE FÍSICA CLÁSSICA':7.9, 'FUNDAMENTOS DE FÍSICA MODERNA':7, 'GERÊNCIA DA INFORMAÇÃO':9.8, 'INTRODUÇÃO A COMPUTAÇÃO':9, 'LAB.DE ESTRUTURA DE DADOS E ALGORITMOS':8.6, 'LABORATÓRIO DE PROGRAMAÇÃO I':8.8, 'LABORATÓRIO DE PROGRAMAÇÃO II':9.3, 'LEITURA E PRODUCAO DE TEXTOS':10, 'LÓGICA MATEMÁTICA':8, 'MATEMÁTICA DISCRETA':9.8,  'PROBABILIDADE E ESTATISTICA':7.3, 'PROGRAMAÇÃO I':8.8, 'PROGRAMAÇÃO II':9, 'SISTEMAS DE INFORMAÇÃO I':8, 'TEORIA DA COMPUTAÇÃO': 8.2, 'TEORIA DOS GRAFOS':10, 'ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA':8.3}

['ALGEBRA LINEAR I', 'ANÁLISE E TÉCNICA DE ALGORITMOS', 'AVAL.DE DESEMPENHO DE SISTEMAS DISCRETOS', 'BANCO DE DADOS I', 'BANCO DE DADOS II', 'CALCULO DIFERENCIAL E INTEGRAL I', 'CALCULO DIFERENCIAL E INTEGRAL II', 'COMPILADORES', 'DIREITO E CIDADANIA', 'ENGENHARIA DE SOFTWARE I', 'ESTRUTURA DE DADOS E ALGORITMOS', 'FUNDAMENTOS DE FÍSICA CLÁSSICA', 'FUNDAMENTOS DE FÍSICA MODERNA', 'GERÊNCIA DA INFORMAÇÃO', 'INFORMÁTICA E SOCIEDADE', 'INTELIGENCIA ARTIFICIAL I', 'INTERCONEXÃO DE REDES DE COMPUTADORES', 'INTRODUÇÃO A COMPUTAÇÃO', 'LAB.DE ESTRUTURA DE DADOS E ALGORITMOS', 'LAB.DE INTERCON.DE REDES DE COMPUTADORES', 'LAB.DE ORG.E ARQUITETURA DE COMPUTADORES', 'LABORATÓRIO DE ENGENHARIA DE SOFTWARE', 'LABORATÓRIO DE PROGRAMAÇÃO I', 'LABORATÓRIO DE PROGRAMAÇÃO II', 'LEITURA E PRODUCAO DE TEXTOS', 'LÓGICA MATEMÁTICA', 'MATEMÁTICA DISCRETA', 'METODOLOGIA CIENTÍFICA', 'METODOS ESTATISTICOS', 'MÉTODOS E SOFTWARE NUMÉRICOS', 'ORG.E ARQUITETURA DE COMPUTADORES I', 'PARADIGMAS DE LING. DE PROGRAMAÇÃ

In [659]:
n_vizinhos = 100
pontos_min_viz = 0.7
n_teste_alunos = 4
n_disci_prox_per = 6
print(COMPUTACAOdf.index)

Int64Index([     0,      1,      3,      4,      5,      7,      8,      9,
                11,     13,
            ...
            442913, 442914, 442915, 442916, 443056, 443057, 443058, 443059,
            443060, 443061],
           dtype='int64', length=33780)


In [64]:
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))
is_number(COMPUTACAOdf.dtypes)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [131]:
prerequisitos = pd.read_csv('pre_requisitos.tsv', sep='\t')
prerequisitos.head()

,codigo_curso,curriculo,codigo_disciplina,codigo_prerequisito
0,13301100,1995,1303002,1303001
1,13301100,1995,1301004,1301001
2,13301100,1995,1109084,1109083
3,13301100,1995,1301063,1301041
4,13301100,1995,1301026,1301005


In [786]:
def get_sim(data_frame):
    data_frame.set_index(data_frame.Matricula) 
    inversa_df = data_frame.transpose()
    resultado = inversa_df[is_number(data_frame)].corr(method='pearson')
    return resultado

def get_neigh(matricula_aluno, df_corr):
    df_corr.at[matricula_aluno, matricula_aluno] = 0
    vizinhos = df_corr[[matricula_aluno,]]

    k_viz_prox = vizinhos.sort_values(by=[matricula_aluno], ascending=False)[1:n_vizinhos+1]
    return k_viz_prox

# como o numero de vizinhos calculado como otimo foi 3(mt pequeno), é necessário prevenir que muitas pessoas
#fiquem sem predição, logo é preciso aumentar a quantidade de vizinhos para aumentar a chance dele ter pago a cadeira em questao
# mas evitar que a precisão da predição diminua por considerar também vizinhos menos semelhantes
def get_nota(data_frame, k_viz_prox, disc_pred):
    notas_vizinhos = data_frame.loc[list(k_vizinhos.index),]
    notas_vizinhos = notas_vizinhos.loc[notas_vizinhos[disc_pred].notnull()]
        
    #se a quantidade de linhas for ==0, ou seja, não tem vizinhos, não há como calcular a nota
    if len(list(notas_vizinhos.index)) == 0:
        return 'NaN'
    
    notas_vizinhos['Similaridade'] = -0.00

    for index, alunos in notas_vizinhos.iterrows():

        notas_vizinhos.at[index,'Similaridade'] = k_viz_prox.at[index,k_viz_prox.columns[0]]

    notas_validas = notas_vizinhos.loc[notas_vizinhos['Similaridade'] > pontos_min_viz]

    
    if(len(notas_validas.index) == 0):
        print('invalida')
        return 'NaN'
    notas_validas = notas_validas.sort_values(by=['Similaridade'], ascending=False)[1:4]
 
    nota_predita = round(sum(notas_validas[disc_pred]*notas_validas['Similaridade'])/ sum(notas_validas['Similaridade']),1)
    return nota_predita

def prox_cadeiras(historico, sem_prerequisito):
    possibilidades = prerequisitos.loc[prerequisitos['codigo_prerequisito'].isin(historico)]
    possibilidades = possibilidades[['codigo_disciplina']]

    for cadeira in possibilidades.codigo_disciplina:
        if not cadeira_paga(cadeira,historico) and cadeira not in sem_prerequisito:
            prerequisitos_cadeira = prerequisitos.loc[prerequisitos['codigo_disciplina'] == cadeira]

            pre=prerequisitos_cadeira['codigo_prerequisito'].isin(historico)
            prerequisitos_pagos = True
            for  c in pre:
                if c == False:
                    prerequisitos_pagos = False
                    break
            if prerequisitos_pagos:
                sem_prerequisito.append(cadeira)
    return sem_prerequisito

def cadeira_paga(cadeira, historico):
    cadeira_isin_historico = False
    for cadeira_paga in historico:
        if cadeira_paga == cadeira:
            cadeira_isin_historico = True
            break
    return cadeira_isin_historico
        
def nomeCadeira_para_cod(conj_cadeiras, cod_nome_df):
    cod_conj_cadeiras = []
    for cadeira in conj_cadeiras:
        aux = cod_nome_df.loc[cod_nome_df['Nome_Disciplina'] == cadeira]
        aux = aux[['Cod_Disciplina']]
        if not aux.empty:
            cod_conj_cadeiras.append(aux.iloc[0]['Cod_Disciplina'])
    return cod_conj_cadeiras

def cod_para_nomeCadeira(conj_cadeiras, cod_nome_df):
    cod_conj_cadeiras = []
    for cadeira in conj_cadeiras:
        aux = cod_nome_df.loc[cod_nome_df['Cod_Disciplina'] == cadeira]
        aux = aux[['Nome_Disciplina']]
    
        if not aux.empty:
            cod_conj_cadeiras.append(aux.iloc[0]['Nome_Disciplina'])
    return cod_conj_cadeiras

for index, aluno in alunos_predicao.iterrows():
    cadeiras_pagas = list(aluno.T.dropna().index)
    prox_possiveis_cadeiras = cod_para_nomeCadeira(prox_cadeiras(cadeiras_pagas, sem_prerequisito),nomeCode_cadeiras)
    k_vizinhos = get_neigh('B'+str(index), correlacao_alunos)

    for cadeira in prox_possiveis_cadeiras:
        print(get_nota(COMPUTACAOtdf.T[[cadeira]],k_vizinhos, cadeira))


8.2
3.5
2.6
5.4
3.0
8.5
8.1
8.7
6.3
7.1
6.5
7.2
7.0
7.1
7.3


In [181]:
cadeiras_cod = COMPUTACAOdf.iloc[:,[1,2]]
cadeiras_code = cadeiras_cod.Cod_Disciplina.unique()
cadeiras_nome = cadeiras_cod.Nome_Disciplina.unique()
nomeCode_cadeiras = pd.DataFrame({"Cod_Disciplina":cadeiras_code, "Nome_Disciplina":cadeiras_nome})
nomeCode_cadeiras.head()

,Cod_Disciplina,Nome_Disciplina
0,1411191,INTERCONEXÃO DE REDES DE COMPUTADORES
1,1411196,INTELIGENCIA ARTIFICIAL I
2,1411173,LÓGICA MATEMÁTICA
3,1109035,ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA
4,1411190,REDES DE COMPUTADORES


In [345]:
sem_prerequisito = ['DIREITO E CIDADANIA','GERÊNCIA DA INFORMAÇÃO','INFORMÁTICA E SOCIEDADE','MATEMÁTICA DISCRETA',
                    'METODOLOGIA CIENTÍFICA','SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)']
sem_prerequisito = nomeCadeira_para_cod(sem_prerequisito, nomeCode_cadeiras)
historico = nomeCadeira_para_cod(meu_historico, nomeCode_cadeiras)
cod_para_nomeCadeira(prox_cadeiras(historico, sem_prerequisito),nomeCode_cadeiras)

['DIREITO E CIDADANIA',
 'GERÊNCIA DA INFORMAÇÃO',
 'INFORMÁTICA E SOCIEDADE',
 'MATEMÁTICA DISCRETA',
 'METODOLOGIA CIENTÍFICA',
 'SEMINÁRIOS (EDUCAÇÃO AMBIENTAL)',
 'ENGENHARIA DE SOFTWARE I',
 'LAB.DE ORG.E ARQUITETURA DE COMPUTADORES',
 'METODOS ESTATISTICOS',
 'ORG.E ARQUITETURA DE COMPUTADORES I',
 'PARADIGMAS DE LING. DE PROGRAMAÇÃO',
 'ANÁLISE E TÉCNICA DE ALGORITMOS',
 'BANCO DE DADOS I',
 'SISTEMAS DE INFORMAÇÃO II',
 'AVAL.DE DESEMPENHO DE SISTEMAS DISCRETOS']

In [806]:
# df_predicao = pd.DataFrame(meu_historico, index=[1])
correlacao_alunos = COMPUTACAOtdf.corr()
alunos_predicao = COMPUTACAOdf
alunos_predicao['Matricula'] = COMPUTACAOdf['Matricula'].map(lambda x: x.lstrip('B'))
alunos_predicao = pd.pivot_table(alunos_predicao, values = 'Media_Disciplina', index = ['Matricula'], columns = 'Nome_Disciplina')



/home/livia.cavalcanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
for i in range(1276):
    alunos_predicao = alunos_predicao.sample(n=4)
    for index, aluno in alunos_predicao.iterrows():
        cadeiras_pagas = list(aluno.T.dropna().index)
        prox_possiveis_cadeiras = cod_para_nomeCadeira(prox_cadeiras(cadeiras_pagas, sem_prerequisito),nomeCode_cadeiras)
        k_vizinhos = get_neigh('B'+str(index), correlacao_alunos)
        dict_notas = {}

        for cadeira in prox_possiveis_cadeiras:
            dict_notas[cadeira] = math.trunc(get_nota(COMPUTACAOtdf.T[[cadeira]],k_vizinhos, cadeira)*10)
        
        print("Notas do historico do aluno:")
        print(aluno.T.dropna())
        print("-----------")
        print("Notas preditas: ")
        for cadeira in dict_notas: 
            print(cadeira, ": ", dict_notas[cadeira]/10)
        print("================")


Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          3.166667
GERÊNCIA DA INFORMAÇÃO                    5.700000
INFORMÁTICA E SOCIEDADE                   7.700000
INTRODUÇÃO A COMPUTAÇÃO                   5.600000
LABORATÓRIO DE PROGRAMAÇÃO I              7.000000
LABORATÓRIO DE PROGRAMAÇÃO II             3.533333
LEITURA E PRODUCAO DE TEXTOS              6.200000
MATEMÁTICA DISCRETA                       2.900000
PROGRAMAÇÃO I                             7.000000
PROGRAMAÇÃO II                            3.166667
TEORIA DOS GRAFOS                         2.500000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    2.100000
Name: 15102, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  5.7
GERÊNCIA DA INFORMAÇÃO :  6.1
INFORMÁTICA E SOCIEDADE :  8.2
MATEMÁTICA DISCRETA :  0.2
METODOLOGIA CIENTÍFICA :  5.8
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.1
ENGENHARIA DE SOFTWARE I :  7.0
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  8.5
METODOS ESTATISTICOS :

Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          3.166667
GERÊNCIA DA INFORMAÇÃO                    5.700000
INFORMÁTICA E SOCIEDADE                   7.700000
INTRODUÇÃO A COMPUTAÇÃO                   5.600000
LABORATÓRIO DE PROGRAMAÇÃO I              7.000000
LABORATÓRIO DE PROGRAMAÇÃO II             3.533333
LEITURA E PRODUCAO DE TEXTOS              6.200000
MATEMÁTICA DISCRETA                       2.900000
PROGRAMAÇÃO I                             7.000000
PROGRAMAÇÃO II                            3.166667
TEORIA DOS GRAFOS                         2.500000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    2.100000
Name: 15102, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  5.7
GERÊNCIA DA INFORMAÇÃO :  6.1
INFORMÁTICA E SOCIEDADE :  8.2
MATEMÁTICA DISCRETA :  0.2
METODOLOGIA CIENTÍFICA :  5.8
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.1
ENGENHARIA DE SOFTWARE I :  7.0
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  8.5
METODOS ESTATISTICOS :

Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          0.150000
GERÊNCIA DA INFORMAÇÃO                    0.000000
INTRODUÇÃO A COMPUTAÇÃO                   2.533333
LABORATÓRIO DE PROGRAMAÇÃO I              2.500000
LEITURA E PRODUCAO DE TEXTOS              2.333333
PROGRAMAÇÃO I                             2.500000
TEORIA DOS GRAFOS                         0.000000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    0.600000
Name: 13613, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  3.0
GERÊNCIA DA INFORMAÇÃO :  1.8
INFORMÁTICA E SOCIEDADE :  5.6
MATEMÁTICA DISCRETA :  1.2
METODOLOGIA CIENTÍFICA :  3.3
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.6
ENGENHARIA DE SOFTWARE I :  7.2
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  6.1
METODOS ESTATISTICOS :  3.0
ORG.E ARQUITETURA DE COMPUTADORES I :  4.4
PARADIGMAS DE LING. DE PROGRAMAÇÃO :  7.8
ANÁLISE E TÉCNICA DE ALGORITMOS :  5.3
BANCO DE DADOS I :  3.8
SISTEMAS DE INFORMAÇÃO II :  4.2
AVAL.DE DESEMPENH

Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          0.150000
GERÊNCIA DA INFORMAÇÃO                    0.000000
INTRODUÇÃO A COMPUTAÇÃO                   2.533333
LABORATÓRIO DE PROGRAMAÇÃO I              2.500000
LEITURA E PRODUCAO DE TEXTOS              2.333333
PROGRAMAÇÃO I                             2.500000
TEORIA DOS GRAFOS                         0.000000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    0.600000
Name: 13613, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  3.0
GERÊNCIA DA INFORMAÇÃO :  1.8
INFORMÁTICA E SOCIEDADE :  5.6
MATEMÁTICA DISCRETA :  1.2
METODOLOGIA CIENTÍFICA :  3.3
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.6
ENGENHARIA DE SOFTWARE I :  7.2
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  6.1
METODOS ESTATISTICOS :  3.0
ORG.E ARQUITETURA DE COMPUTADORES I :  4.4
PARADIGMAS DE LING. DE PROGRAMAÇÃO :  7.8
ANÁLISE E TÉCNICA DE ALGORITMOS :  5.3
BANCO DE DADOS I :  3.8
SISTEMAS DE INFORMAÇÃO II :  4.2
AVAL.DE DESEMPENH

AVAL.DE DESEMPENHO DE SISTEMAS DISCRETOS :  7.4
Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          3.166667
GERÊNCIA DA INFORMAÇÃO                    5.700000
INFORMÁTICA E SOCIEDADE                   7.700000
INTRODUÇÃO A COMPUTAÇÃO                   5.600000
LABORATÓRIO DE PROGRAMAÇÃO I              7.000000
LABORATÓRIO DE PROGRAMAÇÃO II             3.533333
LEITURA E PRODUCAO DE TEXTOS              6.200000
MATEMÁTICA DISCRETA                       2.900000
PROGRAMAÇÃO I                             7.000000
PROGRAMAÇÃO II                            3.166667
TEORIA DOS GRAFOS                         2.500000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    2.100000
Name: 15102, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  5.7
GERÊNCIA DA INFORMAÇÃO :  6.1
INFORMÁTICA E SOCIEDADE :  8.2
MATEMÁTICA DISCRETA :  0.2
METODOLOGIA CIENTÍFICA :  5.8
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.1
ENGENHARIA DE SOFTWARE I :  7.0
LAB.DE ORG.E ARQUITETU

...

Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          0.150000
GERÊNCIA DA INFORMAÇÃO                    0.000000
INTRODUÇÃO A COMPUTAÇÃO                   2.533333
LABORATÓRIO DE PROGRAMAÇÃO I              2.500000
LEITURA E PRODUCAO DE TEXTOS              2.333333
PROGRAMAÇÃO I                             2.500000
TEORIA DOS GRAFOS                         0.000000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    0.600000
Name: 13613, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  3.0
GERÊNCIA DA INFORMAÇÃO :  1.8
INFORMÁTICA E SOCIEDADE :  5.6
MATEMÁTICA DISCRETA :  1.2
METODOLOGIA CIENTÍFICA :  3.3
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.6
ENGENHARIA DE SOFTWARE I :  7.2
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  6.1
METODOS ESTATISTICOS :  3.0
ORG.E ARQUITETURA DE COMPUTADORES I :  4.4
PARADIGMAS DE LING. DE PROGRAMAÇÃO :  7.8
ANÁLISE E TÉCNICA DE ALGORITMOS :  5.3
BANCO DE DADOS I :  3.8
SISTEMAS DE INFORMAÇÃO II :  4.2
AVAL.DE DESEMPENH

Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          0.150000
GERÊNCIA DA INFORMAÇÃO                    0.000000
INTRODUÇÃO A COMPUTAÇÃO                   2.533333
LABORATÓRIO DE PROGRAMAÇÃO I              2.500000
LEITURA E PRODUCAO DE TEXTOS              2.333333
PROGRAMAÇÃO I                             2.500000
TEORIA DOS GRAFOS                         0.000000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    0.600000
Name: 13613, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  3.0
GERÊNCIA DA INFORMAÇÃO :  1.8
INFORMÁTICA E SOCIEDADE :  5.6
MATEMÁTICA DISCRETA :  1.2
METODOLOGIA CIENTÍFICA :  3.3
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.6
ENGENHARIA DE SOFTWARE I :  7.2
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  6.1
METODOS ESTATISTICOS :  3.0
ORG.E ARQUITETURA DE COMPUTADORES I :  4.4
PARADIGMAS DE LING. DE PROGRAMAÇÃO :  7.8
ANÁLISE E TÉCNICA DE ALGORITMOS :  5.3
BANCO DE DADOS I :  3.8
SISTEMAS DE INFORMAÇÃO II :  4.2
AVAL.DE DESEMPENH

Notas do historico do aluno:
Nome_Disciplina
CALCULO DIFERENCIAL E INTEGRAL I          0.150000
GERÊNCIA DA INFORMAÇÃO                    0.000000
INTRODUÇÃO A COMPUTAÇÃO                   2.533333
LABORATÓRIO DE PROGRAMAÇÃO I              2.500000
LEITURA E PRODUCAO DE TEXTOS              2.333333
PROGRAMAÇÃO I                             2.500000
TEORIA DOS GRAFOS                         0.000000
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    0.600000
Name: 13613, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  3.0
GERÊNCIA DA INFORMAÇÃO :  1.8
INFORMÁTICA E SOCIEDADE :  5.6
MATEMÁTICA DISCRETA :  1.2
METODOLOGIA CIENTÍFICA :  3.3
SEMINÁRIOS (EDUCAÇÃO AMBIENTAL) :  8.6
ENGENHARIA DE SOFTWARE I :  7.2
LAB.DE ORG.E ARQUITETURA DE COMPUTADORES :  6.1
METODOS ESTATISTICOS :  3.0
ORG.E ARQUITETURA DE COMPUTADORES I :  4.4
PARADIGMAS DE LING. DE PROGRAMAÇÃO :  7.8
ANÁLISE E TÉCNICA DE ALGORITMOS :  5.3
BANCO DE DADOS I :  3.8
SISTEMAS DE INFORMAÇÃO II :  4.2
AVAL.DE DESEMPENH

Notas do historico do aluno:
Nome_Disciplina
ALGEBRA LINEAR I                          5.10
CALCULO DIFERENCIAL E INTEGRAL I          7.10
CALCULO DIFERENCIAL E INTEGRAL II         3.70
DIREITO E CIDADANIA                       9.50
FUNDAMENTOS DE FÍSICA CLÁSSICA            5.60
FUNDAMENTOS DE FÍSICA MODERNA             9.30
GERÊNCIA DA INFORMAÇÃO                    3.85
INFORMÁTICA E SOCIEDADE                   8.30
INTRODUÇÃO A COMPUTAÇÃO                   5.10
LABORATÓRIO DE PROGRAMAÇÃO I              3.85
LABORATÓRIO DE PROGRAMAÇÃO II             4.60
LEITURA E PRODUCAO DE TEXTOS              7.40
MATEMÁTICA DISCRETA                       4.70
PROBABILIDADE E ESTATISTICA               5.00
PROGRAMAÇÃO I                             4.30
PROGRAMAÇÃO II                            5.30
TEORIA DOS GRAFOS                         5.00
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    7.80
Name: 2129, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  9.3
GERÊNCIA DA INFORMAÇÃO :  

Notas do historico do aluno:
Nome_Disciplina
ALGEBRA LINEAR I                          5.10
CALCULO DIFERENCIAL E INTEGRAL I          7.10
CALCULO DIFERENCIAL E INTEGRAL II         3.70
DIREITO E CIDADANIA                       9.50
FUNDAMENTOS DE FÍSICA CLÁSSICA            5.60
FUNDAMENTOS DE FÍSICA MODERNA             9.30
GERÊNCIA DA INFORMAÇÃO                    3.85
INFORMÁTICA E SOCIEDADE                   8.30
INTRODUÇÃO A COMPUTAÇÃO                   5.10
LABORATÓRIO DE PROGRAMAÇÃO I              3.85
LABORATÓRIO DE PROGRAMAÇÃO II             4.60
LEITURA E PRODUCAO DE TEXTOS              7.40
MATEMÁTICA DISCRETA                       4.70
PROBABILIDADE E ESTATISTICA               5.00
PROGRAMAÇÃO I                             4.30
PROGRAMAÇÃO II                            5.30
TEORIA DOS GRAFOS                         5.00
ÁLGEBRA VETORIAL E GEOMETRIA ANALÍTICA    7.80
Name: 2129, dtype: float64
-----------
Notas preditas: 
DIREITO E CIDADANIA :  9.3
GERÊNCIA DA INFORMAÇÃO :  

In [805]:
meu_historico
dt = pd.DataFrame(list(meu_historico.items())).T
COMPUTACAOtdf.append(dt)
COMPUTACAOtdf.head()

Matricula,B10079,B10082,B10083,B10086,B10087,B10088,B10092,B1012,B10172,B10192,...,B9834,B9866,B9871,B9873,B9875,B9877,B9879,B9881,B99,B9950
Nome_Disciplina,,,,,,,,,,,,,,,,,,,,,
ALGEBRA LINEAR I,NaN,NaN,0.0,NaN,NaN,NaN,6.1,1.95,1.75,NaN,...,NaN,NaN,NaN,7.1,NaN,5.5,NaN,NaN,7.4,NaN
ANÁLISE E TÉCNICA DE ALGORITMOS,NaN,6.2,NaN,NaN,NaN,NaN,7.0,7.10,NaN,6.2,...,NaN,NaN,NaN,7.0,NaN,7.8,NaN,NaN,7.0,NaN
AVAL.DE DESEMPENHO DE SISTEMAS DISCRETOS,NaN,NaN,NaN,NaN,NaN,NaN,8.1,NaN,NaN,7.3,...,NaN,NaN,NaN,7.6,NaN,7.3,NaN,NaN,8.7,NaN
BANCO DE DADOS I,NaN,NaN,0.0,NaN,NaN,NaN,7.6,5.30,NaN,4.0,...,NaN,NaN,NaN,7.6,NaN,7.0,NaN,NaN,7.0,NaN
BANCO DE DADOS II,NaN,NaN,NaN,NaN,NaN,NaN,5.4,NaN,NaN,6.8,...,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,6.5,NaN
